# HW1 - part 4
<a id=part4></a>

In [31]:
import sys, random,binascii,AES,jupyter_utils,random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display,HTML
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150


This section will focus on the Diffusion principle and examine the extent to which a change in input or internal states affects the end result. The runs in the strain part will be performed on a single input block (16 characters)

In [32]:
plaintext = "From the Pope to"
print("Total Length:",len(plaintext))
display(Markdown(f'\n"**_{plaintext}_**"'))


Total Length: 16



"**_From the Pope to_**"

And we will try to encrypt the message, using an encryption key **"R is a Short key"**
 

In [33]:
key = "R is a Short key"
key = key.encode().hex()
print(f"'R is a Short key':\n 0x{key}")
aes_lib = AES.AES(int(key,base=16))

'R is a Short key':
 0x5220697320612053686f7274206b6579


## Question 4:
1. Run the AES algorithm, and save the intermediate results after each round (including the final result)
2. Randomly select a bit in the login message and change it from 0 -> 1 or 1-> 0 accordingly Repeat (1) and calculate how many bits changed their value as a result of the change of the single bit in each round, and the encrypted message (relative to the same situation when no errors were injected)
4. Repeat the experiment 5 times when each time you change one bit in relation to the message in (1) at the end of the runs, present a table

In [34]:
def run_aes_stages(plaintext):
    spec_enc_text = [""]*10
    for i in range(0, len(plaintext), 16):  
        ciper = aes_lib.encrypt_by_stage(int(plaintext[i:i+16].encode().hex(),base=16))
        for j,c in enumerate(ciper):
            spec_enc_text[j] += ('{:x}'.format(c)).zfill(32)
    return spec_enc_text
spec_enc_text = run_aes_stages(plaintext)
display(spec_enc_text)

['cb95e6a2a56a602164f8bdc338622f85',
 'cbf42cb3f27bbb6266e1e7abbc87199e',
 '5953b07e2a304dc27be50e6355176310',
 '203d5627e023761e3d86efd4ace38e38',
 'b874b055f6f6fc733e8c7952dbf7ea9c',
 '1dba47880ed16f3c72d966c2526347cc',
 '737bf4f4317604824733b3b13695b2cf',
 'd0d5082d7e23c65c2e2cd9304833f9f6',
 '54d92855bb1e7c8075c0eb813950e85d',
 '793ec12a27911757630e1fd3c4affd06']

For stage II and on, we wants' to create a convertion function

In [35]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8 ):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def count_flips_to_convert(a, b):

    diff = a ^ b

    # count number of ones in diff
    count = 0
    while diff:
        diff &= (diff - 1)
        count += 1
    return count

In [36]:
def flip_plaintext(plaintext):
    bitflip_p = tobits(plaintext)
    place = random.choice(range(len(bitflip_p)))
    bitflip_p[place] = int(1 - bitflip_p[place])
    bitflip_plaintext = frombits(bitflip_p)
    return bitflip_plaintext,place
bitflip_plaintext,place = flip_plaintext(plaintext)
print("The bit which was chosen to be replaced: ", place)
print("Result:",bitflip_plaintext)

The bit which was chosen to be replaced:  84
Result: From the Pgpe to


In [37]:
bitflip_spec_enc_text = run_aes_stages(bitflip_plaintext)
display(bitflip_spec_enc_text)

['3689075fa56a602164f8bdc338622f85',
 'ffee369d078ebf932d3c71e030869413',
 '420575864268b0954142c48e54fdd4db',
 'f2e910056962f56de0f60ce978c6e11a',
 'cf8fb4eb8a7dc9e968d83e414287a3a0',
 '2c0f567ce17e68dafdff92ecb51b77f9',
 '2079799e7348b9b23e000eab1c980209',
 '53e34bc4e144691793d9d92bcdc2413a',
 'f0fcd54bbf775050bcde28a772b98bd9',
 'd5b91c53c536b1189b0b284d962abe56']

In [38]:
def calulate_diffrence(textlist,flip_textlist):
    diff = []
    for i in range(len(flip_textlist)):
        p =  int(textlist[i],base=16)
        c =  int(flip_textlist[i],base=16)
        diff.append(count_flips_to_convert(p,c))
    return diff
diff = calulate_diffrence(bitflip_spec_enc_text,spec_enc_text)
display(diff)

[21, 61, 71, 60, 65, 69, 59, 68, 58, 65]

In [39]:
table_result = np.zeros((6,len(bitflip_spec_enc_text)))

for i in range(5):
    bfp,place = flip_plaintext(plaintext)
    bfp_stage = run_aes_stages(bfp)
    table_result[i] = calulate_diffrence(bfp_stage,spec_enc_text)


In [40]:
table_result[5] = np.mean(table_result[:5], axis = 0)
display(pd.DataFrame(table_result, columns=[f"Round {col+1}" for col in range(len(bitflip_spec_enc_text))]))

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,Round 9,Round 10
0,20.0,62.0,67.0,64.0,57.0,68.0,71.0,67.0,53.0,63.0
1,21.0,66.0,56.0,67.0,57.0,73.0,58.0,70.0,64.0,65.0
2,15.0,65.0,48.0,60.0,59.0,63.0,73.0,73.0,56.0,71.0
3,20.0,67.0,62.0,58.0,56.0,65.0,64.0,67.0,64.0,54.0
4,68.0,61.0,64.0,71.0,53.0,67.0,60.0,61.0,69.0,62.0
5,28.8,64.2,59.4,64.0,56.4,67.2,65.2,67.6,61.2,63.0
